In [3]:
import pandas as pd
import numpy as np

In [4]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,FlightNumber,Origin,Destination,Date,Seats
0,FL001,JFK,LAX,2023-06-10,199
1,FL002,LAX,JFK,2023-06-12,149
2,FL003,JFK,SFO,2023-06-10,179
3,FL004,SFO,JFK,2023-06-15,220
4,FL005,JFK,ORD,2023-06-11,190


In [5]:
bookings = pd.read_csv('data/bookings.csv')
bookings.head()

,BookingID,FlightNumber,SeatNumber
0,B001,FL001,12
1,B002,FL002,8
2,B003,FL003,15


In [6]:
airports = set(flights['Origin'])
airports.update(flights['Destination'])
airports

{'ATL',
 'BOS',
 'BWI',
 'CLT',
 'DCA',
 'DEN',
 'DFW',
 'FLL',
 'HNL',
 'IAD',
 'IAH',
 'JFK',
 'LAS',
 'LAX',
 'MCO',
 'MIA',
 'MSP',
 'ORD',
 'PDX',
 'PHX',
 'SAN',
 'SEA',
 'SFO',
 'SJC',
 'TPA'}

# Flight Search

In [7]:
def flight_search():
    while True:
        origin = input("Origin? ")
        destination = input("Destination? ")
        date = input("Travel date? (YYYY-MM-DD) ")
        
        if origin not in airports or destination not in airports:
            print("Invalid airport code.")
            continue
        
        try:
            pd.to_datetime(date)
        except ValueError:
            print("Invalid date format. Please enter a date in the format YYYY-MM-DD.")
            continue
        
        matching_flights = flights[
            (flights['Origin'] == origin.upper()) &
            (flights['Destination'] == destination.upper()) &
            (flights['Date'] == date)
        ]
        
        if matching_flights.empty:
            print("No flights!!")
        else:
            print("Available flights:")
            print(matching_flights)
            break

In [8]:
flight_search()

No flights!!
Available flights:
  FlightNumber Origin Destination        Date  Seats
0        FL001    JFK         LAX  2023-06-10    199


# Flight Selection

In [9]:
selected_flight = None

def flight_selection():
    global selected_flight
    
    while True:
        flight_number = input("Flight number? ")
        
        if flight_number not in flights['FlightNumber'].values:
            print("Invalid flight number!")
            continue
        
        selected_flight = flights[flights['FlightNumber'] == flight_number]
        
        if selected_flight.empty:
            print("No flight found for the given flight number. Please try again.")
        else:
            print("Selected flight:")
            print(selected_flight)
            break


In [33]:
flight_selection()

Selected flight:
  FlightNumber Origin Destination        Date  Seats
3        FL004    SFO         JFK  2023-06-15    220


In [11]:
def booking_confirmation():
    print('Booking Confirmation:')
    print(bookings.iloc[-1])
    print(flights[flights['FlightNumber']==bookings.iloc[-1]['FlightNumber']])

In [31]:
def seat_selection():
    global selected_flight
    global flights
    global bookings
    
    if selected_flight is None:
        print("No flight has been selected.")
        return
    
    flight_number = selected_flight['FlightNumber'].values[0]
    seat_number = input("Seat number? ")
    
    if bookings[(bookings['FlightNumber'] == flight_number) & (bookings['SeatNumber'] == seat_number)].shape[0] > 0:
        print("Seat is already booked. Please choose another seat.")
        return
    
    booking_details = pd.DataFrame({
        'BookingID': bookings['BookingID'].values[-1][:-1] + str(int(bookings['BookingID'].values[-1][-1]) + 1),
        'FlightNumber': flight_number,
        'SeatNumber': seat_number,
    }, index=[bookings.size//3])
    
    bookings = pd.concat([bookings, booking_details])
    print("Booked!")
    booking_confirmation()

    flights.loc[flights['FlightNumber'] == flight_number, 'Seats'] -= 1

In [39]:
seat_selection()

Booked!
Booking Confirmation:
BookingID       B0013
FlightNumber    FL004
SeatNumber          6
Name: 11, dtype: object
  FlightNumber Origin Destination        Date  Seats
3        FL004    SFO         JFK  2023-06-15    215


In [14]:
flights[flights['FlightNumber']==selected_flight['FlightNumber'].values[0]]

,FlightNumber,Origin,Destination,Date,Seats
0,FL001,JFK,LAX,2023-06-10,198


In [27]:
def view_bookings():
    print(bookings)
view_bookings()

  BookingID FlightNumber SeatNumber
1      B002        FL002          8
2      B003        FL003         15
3      B004        FL001          1
3      B005        FL001          2
4      B006        FL001          3


In [16]:
def cancel_booking():
    global bookings
    global flights

    booking_id = input('Booking ID? ')
    
    if booking_id not in bookings['BookingID'].values:
        print("Invalid Booking ID!")
        return
    
    flight_number = bookings.loc[bookings['BookingID'] == booking_id, 'FlightNumber'].values[0]
    seat_number = bookings.loc[bookings['BookingID'] == booking_id, 'SeatNumber'].values[0]
    
    flights.loc[flights['FlightNumber'] == flight_number, 'Seats'] += 1
    bookings = bookings[bookings['BookingID'] != booking_id]
    
    print("Booking cancelled successfully.")
    print(f"Flight number: {flight_number}")
    print(f"Seat number: {seat_number}")

In [17]:
cancel_booking()

Booking cancelled successfully.
Flight number: FL001
Seat number: 12


In [40]:
bookings

,BookingID,FlightNumber,SeatNumber
1,B002,FL002,8
2,B003,FL003,15
3,B004,FL001,1
3,B005,FL001,2
4,B006,FL001,3
5,B007,FL001,4
6,B008,FL004,1
7,B009,FL004,2
8,B0010,FL004,3
9,B0011,FL004,4


In [19]:
flights.head()

,FlightNumber,Origin,Destination,Date,Seats
0,FL001,JFK,LAX,2023-06-10,199
1,FL002,LAX,JFK,2023-06-12,149
2,FL003,JFK,SFO,2023-06-10,179
3,FL004,SFO,JFK,2023-06-15,220
4,FL005,JFK,ORD,2023-06-11,190


In [20]:
def update_seat_number():
    global bookings
    
    booking_id = input("Booking ID? ")
    
    if booking_id not in bookings['BookingID'].values:
        print("Invalid Booking ID")
        return
    
    new_seat_number = input("New seat number? ")
    flight_number = bookings.loc[bookings['BookingID'] == booking_id,  'FlightNumber'].values[0]

    if bookings[(bookings['BookingID'] != booking_id) & (bookings['FlightNumber'] == flight_number) & (bookings['SeatNumber'] == new_seat_number)].shape[0] > 0:
        print("Seat number is already booked. Please choose another seat.")
        return
    
    bookings.loc[bookings['BookingID'] == booking_id, 'SeatNumber'] = new_seat_number
    
    print("Seat number updated successfully.")

In [21]:
update_seat_number()

Invalid Booking ID


In [22]:
bookings

,BookingID,FlightNumber,SeatNumber
1,B002,FL002,8
2,B003,FL003,15
3,B004,FL001,1


In [58]:
def flight_report():
    global selected_flight

    flight_number = selected_flight['FlightNumber'].values[0]
    booked_seats = bookings[bookings['FlightNumber']==flight_number].shape[0]
    total_seats = flights.loc[flights['FlightNumber']==flight_number, 'Seats'].values[0]

    flight_occupancy_rate = (booked_seats*100)/(booked_seats + total_seats)
    print('Flight Occupancy Rate of',flight_number,'is',flight_occupancy_rate,'%')
    print('Most popular route:',bookings['FlightNumber'].value_counts().index[0])


In [59]:
flight_report()

Flight Occupancy Rate of FL004 is 2.727272727272727 %
Most popular route: FL004
